# Imports

In [1]:
import matplotlib.pyplot as plt
from generate import read_tfrecord_files
import numpy as np

In [2]:
dataset = read_tfrecord_files()

In [3]:
d = dataset.take(1)
for i in d:
    print(i[0].numpy())

[[[0.08627451 0.08627451 0.08627451]
  [0.08235294 0.08235294 0.08235294]
  [0.07843138 0.07843138 0.07843138]
  ...
  [0.07843138 0.07843138 0.07843138]
  [0.07843138 0.07843138 0.07843138]
  [0.07450981 0.07450981 0.07450981]]

 [[0.07843138 0.07843138 0.07843138]
  [0.07843138 0.07843138 0.07843138]
  [0.07843138 0.07843138 0.07843138]
  ...
  [0.07843138 0.07843138 0.07843138]
  [0.07843138 0.07843138 0.07843138]
  [0.07450981 0.07450981 0.07450981]]

 [[0.07450981 0.07450981 0.07450981]
  [0.07450981 0.07450981 0.07450981]
  [0.07843138 0.07843138 0.07843138]
  ...
  [0.07843138 0.07843138 0.07843138]
  [0.07843138 0.07843138 0.07843138]
  [0.07843138 0.07843138 0.07843138]]

 ...

 [[0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  ...
  [0.07843138 0.07843138 0.07843138]
  [0.07843138 0.07843138 0.07843138]
  [0.07843138 0.07843138 0.07843138]]

 [[0.         0.         0.        ]
  [0.         0.         0.        ]


In [28]:
num = 1026
d = dataset.take(num)
list_names = []
for i in d:
    list_names += [i[2].numpy()]
if len(set(list_names)) == len(list_names):
    print("Equal " + str(len(list_names)))

Equal 1026
